In [1]:
import numpy as np 
import pandas as pd
import re
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from scipy import special
from tqdm import tqdm
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import SimpleImputer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, classification_report, accuracy_score, roc_curve
import matplotlib.pyplot as plt
import optuna
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif



warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20,.2f}'.format)


In [9]:
X_train = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/X_train_corr.csv')
y_train = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/y_train_large.csv').squeeze()
X_test = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/X_test_corr.csv')
y_test = pd.read_csv('/Users/sakshamjain/Desktop/Projects/JAIN-WIN/y_test_large.csv').squeeze()

In [10]:
X_train.head()

,age,bmi,elective_surgery,gender,height,icu_id,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_invasive_max,d1_mbp_invasive_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_invasive_max,h1_diasbp_invasive_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_invasive_max,h1_mbp_invasive_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_invasive_max,h1_sysbp_invasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_albumin_max,d1_albumin_min,d1_bilirubin_max,d1_bilirubin_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_inr_max,d1_inr_min,d1_lactate_max,d1_lactate_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,h1_albumin_max,h1_albumin_min,h1_bilirubin_max,h1_bilirubin_min,h1_bun_max,h1_bun_min,h1_calcium_max,h1_calcium_min,h1_creatinine_max,h1_creatinine_min,h1_glucose_max,h1_glucose_min,h1_hco3_max,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,icu_id_counts,age_counts,nan_counts,sq_age,sq_bmi,bmi_age,weight_age,comorbidity_score,diabete_risk,gcs_sum,apache_2_diagnosis_type,bmi_type,height_type,weight_type,age_type,gcs_sum_type,apache_3j_diagnosis_split1,apache_2_diagnosis_split1,profile,diff_bmi,bilirubin_d1_h1_max_eq,bilirubin_d1_h1_min_eq,bilirubin_h1_value_range,bilirubin_h1_zero_range,bilirubin_h1_mean,bilirubin_h1_std,bilirubin_h1_max_apache_3j_diagnosis_mean,bilirubin_h1_max_apache_3j_diagnosis_diff,bilirubin_h1_max_apache_3j_diagnosis_std,bilirubin_h1_max_apache_3j_diagnosis_norm_std,bilirubin_h1_max_apache_3j_diagnosis_rank,bilirubin_h1_max_apache_3j_diagnosis_count,bilirubin_h1_max_apache_3j_diagnosis_norm_rank,bilirubin_h1_max_apache_3j_diagnosis_skew,bilirubin_h1_min_apache_3j_diagnosis_mean,bilirubin_h1_min_apache_3j_diagnosis_diff,bilirubin_h1_min_apache_3j_diagnosis_std,bilirubin_h1_min_apache_3j_diagnosis_norm_std,bilirubin_h1_min_apache_3j_diagnosis_rank,bilirubin_h1_min_apache_3j_diagnosis_norm_rank,bilirubin_h1_min_apache_3j_diagnosis_skew,bilirubin_h1_max_profile_mean,bilirubin_h1_max_profile_diff,bilirubin_h1_max_profile_std,bilirubin_h1_max_profile_norm_std,bilirubin_h1_max_profile_rank,bilirubin_h1_max_profile_count,bilirubin_h1_max_profile_norm_ran

In [11]:
def evaluate_lightgbm(X_train, y_train, X_test, y_test):
    """
    Returns:
    - model (lgb.LGBMClassifier): Trained LightGBM model.
    - roc_auc (float): ROC-AUC score on the test set.
    - report (str): Classification report with precision, recall, and F1-score up to 4 decimals.
    """
    # Initialize the LightGBM classifier with default parameters
    model = lgb.LGBMClassifier(
        random_state=69,        # For reproducibility
        n_jobs=-1,
        force_col_wise=True              # Utilize all available cores
    )
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Predict probabilities for the positive class
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Predict class labels
    y_pred = model.predict(X_test)
    
    # Calculate ROC-AUC score
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Generate classification report with 4 decimal places
    report = classification_report(y_test, y_pred, digits=4)
    
    # Print the evaluation metrics
    print(f"ROC-AUC: {roc_auc:.8f}")
    print("Classification Report:\n", report)
    
    return model, roc_auc, report

In [12]:
model, auc, report = evaluate_lightgbm(X_train, y_train, X_test, y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 850113
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 3698
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
ROC-AUC: 0.87206368
Classification Report:
               precision    recall  f1-score   support

         0.0     0.8718    0.9385    0.9039     30574
         1.0     0.6934    0.5020    0.5824      8474

    accuracy                         0.8438     39048
   macro avg     0.7826    0.7202    0.7431     39048
weighted avg     0.8331    0.8438    0.8341     39048



In [13]:
def univariate_anova_selection(X_train, X_test, y_train, k=1000):
    """
    Selects top 'k' features based on the ANOVA F-test.

    Parameters:
    - X_train (pd.DataFrame): Training feature set.
    - X_test (pd.DataFrame): Testing feature set.
    - y_train (pd.Series or np.ndarray): Training target variable.
    - k (int): Number of top features to select based on ANOVA F-test.

    Returns:
    - X_train_reduced (pd.DataFrame): Reduced training feature set with top 'k' features.
    - X_test_reduced (pd.DataFrame): Reduced testing feature set with top 'k' features.
    - selected_features (list): List of selected feature names.
    """
    print(f"\nApplying Univariate ANOVA F-test Feature Selection with k={k}...")
    
    # Initialize SelectKBest with ANOVA F-test
    selector = SelectKBest(score_func=f_classif, k=k)
    
    # Fit the selector to the training data
    selector.fit(X_train, y_train)
    
    # Get the boolean mask of selected features
    mask = selector.get_support()
    
    # Extract the selected feature names
    selected_features = X_train.columns[mask].tolist()
    
    # Calculate number of features removed
    num_features_removed = X_train.shape[1] - len(selected_features)
    
    print(f"ANOVA F-test: Removed {num_features_removed} features.")
    print(f"Number of features before: {X_train.shape[1]}")
    print(f"Number of features after: {len(selected_features)}")
    
    # Transform both training and testing data
    X_train_reduced = X_train[selected_features]
    X_test_reduced = X_test[selected_features]
    
    return X_train_reduced, X_test_reduced, selected_features

In [14]:
def univariate_mi_selection(X_train, X_test, y_train, k=1000, discrete_features='auto', random_state=42):
    """
    Selects top 'k' features based on Mutual Information.

    Parameters:
    - X_train (pd.DataFrame): Training feature set.
    - X_test (pd.DataFrame): Testing feature set.
    - y_train (pd.Series or np.ndarray): Training target variable.
    - k (int): Number of top features to select based on Mutual Information.
    - discrete_features (str or array-like, optional): Indicates which features are discrete. 
      - If 'auto', it attempts to determine automatically.
    - random_state (int, optional): Seed used by the random number generator for reproducibility.

    Returns:
    - X_train_reduced (pd.DataFrame): Reduced training feature set with top 'k' features.
    - X_test_reduced (pd.DataFrame): Reduced testing feature set with top 'k' features.
    - selected_features (list): List of selected feature names.
    """
    print(f"\nApplying Univariate Mutual Information Feature Selection with k={k}...")
    
    # Initialize SelectKBest with Mutual Information
    selector = SelectKBest(score_func=mutual_info_classif, k=k)
    
    # Fit the selector to the training data
    selector.fit(X_train, y_train)
    
    # Get the boolean mask of selected features
    mask = selector.get_support()
    
    # Extract the selected feature names
    selected_features = X_train.columns[mask].tolist()
    
    # Calculate number of features removed
    num_features_removed = X_train.shape[1] - len(selected_features)
    
    print(f"Mutual Information: Removed {num_features_removed} features.")
    print(f"Number of features before: {X_train.shape[1]}")
    print(f"Number of features after: {len(selected_features)}")
    
    # Transform both training and testing data
    X_train_reduced = X_train[selected_features]
    X_test_reduced = X_test[selected_features]
    
    return X_train_reduced, X_test_reduced, selected_features


In [15]:
def combined_anova_mi_selection(X_train, X_test, y_train, k=1000):
    """
    Selects top 'k' features based on a combination of ANOVA F-test and Mutual Information scores.
    """
    # Compute ANOVA scores
    anova_selector = SelectKBest(score_func=f_classif, k=k)
    anova_selector.fit(X_train, y_train)
    anova_scores = anova_selector.scores_

    # Compute MI scores
    mi_selector = SelectKBest(score_func=mutual_info_classif, k=k)
    mi_selector.fit(X_train, y_train)
    mi_scores = mi_selector.scores_

    # Combine scores (e.g., by averaging)
    combined_scores = (anova_scores + mi_scores) / 2

    # Select top 'k' features based on combined scores
    top_k_indices = np.argsort(combined_scores)[-k:]
    selected_features = X_train.columns[top_k_indices].tolist()

    # Transform the datasets
    X_train_reduced = X_train[selected_features]
    X_test_reduced = X_test[selected_features]

    print(f"Combined ANOVA and MI: Selected top {k} features.")
    return X_train_reduced, X_test_reduced, selected_features

In [16]:
print(f"Initial Training Features: {X_train.shape}")
print(f"Initial Testing Features: {X_test.shape}")

Initial Training Features: (91109, 3698)
Initial Testing Features: (39048, 3698)


In [17]:
def determine_k(X_train, percentage=0.8):
    """
    Determines the number of top features to select based on a percentage of the current feature set.
    
    Returns:
    - k (int): Number of top features to select.
    """
    total_features = X_train.shape[1]
    k = int(total_features * percentage)
    print(f"Determined k={k} based on {percentage*100}% of {total_features} features.")
    return k

In [18]:
percentage = 0.8
k = determine_k(X_train, percentage=percentage)

X_train_anova, X_test_anova, anova_selected_features = univariate_anova_selection(X_train, X_test, y_train, k=k)
print("\nEvaluating model after ANOVA F-test Feature Selection:")

model_anova, auc_anova, report_anova = evaluate_lightgbm(X_train_anova, y_train, X_test_anova, y_test)

Determined k=2958 based on 80.0% of 3698 features.

Applying Univariate ANOVA F-test Feature Selection with k=2958...
ANOVA F-test: Removed 740 features.
Number of features before: 3698
Number of features after: 2958

Evaluating model after ANOVA F-test Feature Selection:
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 695621
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 2958
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
ROC-AUC: 0.87160354
Classification Report:
               precision    recall  f1-score   support

         0.0     0.8705    0.9365    0.9023     30574
         1.0     0.6846    0.4972    0.5760      8474

    accuracy                         0.8412     39048
   macro avg     0.7775    0.7168    0.7392     3

In [19]:
percentage = 0.9
k = determine_k(X_train, percentage=percentage)

X_train_anova, X_test_anova, anova_selected_features = univariate_anova_selection(X_train, X_test, y_train, k=k)
print("\nEvaluating model after ANOVA F-test Feature Selection:")

model_anova, auc_anova, report_anova = evaluate_lightgbm(X_train_anova, y_train, X_test_anova, y_test)

Determined k=3328 based on 90.0% of 3698 features.

Applying Univariate ANOVA F-test Feature Selection with k=3328...
ANOVA F-test: Removed 370 features.
Number of features before: 3698
Number of features after: 3328

Evaluating model after ANOVA F-test Feature Selection:
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 771895
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 3328
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
ROC-AUC: 0.87115194
Classification Report:
               precision    recall  f1-score   support

         0.0     0.8708    0.9379    0.9031     30574
         1.0     0.6897    0.4978    0.5782      8474

    accuracy                         0.8424     39048
   macro avg     0.7802    0.7178    0.7407     3

In [20]:
percentage = 0.95
k = determine_k(X_train, percentage=percentage)

X_train_anova, X_test_anova, anova_selected_features = univariate_anova_selection(X_train, X_test, y_train, k=k)
print("\nEvaluating model after ANOVA F-test Feature Selection:")

model_anova, auc_anova, report_anova = evaluate_lightgbm(X_train_anova, y_train, X_test_anova, y_test)

Determined k=3513 based on 95.0% of 3698 features.

Applying Univariate ANOVA F-test Feature Selection with k=3513...
ANOVA F-test: Removed 185 features.
Number of features before: 3698
Number of features after: 3513

Evaluating model after ANOVA F-test Feature Selection:
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 19677, number of negative: 71432
[LightGBM] [Info] Total Bins 809420
[LightGBM] [Info] Number of data points in the train set: 91109, number of used features: 3513
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.215972 -> initscore=-1.289296
[LightGBM] [Info] Start training from score -1.289296
ROC-AUC: 0.87120023
Classification Report:
               precision    recall  f1-score   support

         0.0     0.8708    0.9374    0.9029     30574
         1.0     0.6881    0.4980    0.5778      8474

    accuracy                         0.8421     39048
   macro avg     0.7794    0.7177    0.7403     3

In [21]:
percentage = 0.8
k = determine_k(X_train, percentage=percentage)

X_train_mi, X_test_mi, mi_selected_features = univariate_mi_selection(X_train, X_test, y_train, k=k)

print("\nEvaluating model after Mutual Information Feature Selection:")
model_mi, auc_mi, report_mi = evaluate_lightgbm(X_train_mi, y_train, X_test_mi, y_test)

Determined k=2958 based on 80.0% of 3698 features.

Applying Univariate Mutual Information Feature Selection with k=2958...


KeyboardInterrupt: 

: 

In [ ]:
percentage = 0.9
k = determine_k(X_train, percentage=percentage)

X_train_mi, X_test_mi, mi_selected_features = univariate_mi_selection(X_train, X_test, y_train, k=k)

print("\nEvaluating model after Mutual Information Feature Selection:")
model_mi, auc_mi, report_mi = evaluate_lightgbm(X_train_mi, y_train, X_test_mi, y_test)

In [ ]:
percentage = 0.95
k = determine_k(X_train, percentage=percentage)

X_train_mi, X_test_mi, mi_selected_features = univariate_mi_selection(X_train, X_test, y_train, k=k)

print("\nEvaluating model after Mutual Information Feature Selection:")
model_mi, auc_mi, report_mi = evaluate_lightgbm(X_train_mi, y_train, X_test_mi, y_test)

In [ ]:
percentage = 0.8
k = determine_k(X_train, percentage=percentage)

X_train_miano, X_test_miano, miano_selected_features = combined_anova_mi_selection(X_train, X_test, y_train, k=k)
    
print("\nEvaluating model after Mutual Information Feature Selection:")
model_miano, auc_miano, report_miano = evaluate_lightgbm(X_train_miano, y_train, X_test_miano, y_test)

In [ ]:
percentage = 0.9
k = determine_k(X_train, percentage=percentage)

X_train_miano, X_test_miano, miano_selected_features = combined_anova_mi_selection(X_train, X_test, y_train, k=k)
    
print("\nEvaluating model after Mutual Information Feature Selection:")
model_miano, auc_miano, report_miano = evaluate_lightgbm(X_train_miano, y_train, X_test_miano, y_test)

In [ ]:
percentage = 0.95
k = determine_k(X_train, percentage=percentage)

X_train_miano, X_test_miano, miano_selected_features = combined_anova_mi_selection(X_train, X_test, y_train, k=k)
    
print("\nEvaluating model after Mutual Information Feature Selection:")
model_miano, auc_miano, report_miano = evaluate_lightgbm(X_train_miano, y_train, X_test_miano, y_test)